In [ ]:
import pandas
import os

In [ ]:
import utils

In [3]:
import os
from math import radians, cos, sin, asin, sqrt
import sys
import time

import sys
sys.path.append('/Users/gustaveronteix/Documents/Projets/Projets Code/batch7_beges/datasets/chorus-dt')

import requests
import dill
import urllib.parse
import pandas as pd
from tqdm import tqdm
from utils.config import CONFIG

from utils import dataset
#from utils.resolvers import HardcodesResolver, GeocodingApiResolver

tqdm.pandas()


def compute_distance(x):
    lon1 = x["coords_place_0_lon"]
    lat1 = x["coords_place_0_lat"]
    lon2 = x["coords_place_1_lon"]
    lat2 = x["coords_place_1_lat"]
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * asin(sqrt(a))
    r = 6371  # Radius of earth in miles. Use 6371 for kilometers
    return c * r


if __name__ == "__main__":
    data = dataset.load_data("./data/raw/sample-avion-train.csv")
    places, trips = dataset.get_places_and_trips(data, ["T", "TC", "TCA", "TCM", "TU"])

    # Splitting codes / name
    places[["code_1", "name", "temp", "code_2"]] = places["place"].str.extract(
        "([^-]*)\s-\s([^\[]*)\s?([\[](.*)[\]])?", expand=True
    )
    places.drop(columns=["temp"], inplace=True)

    places["resolved"] = False
    places["resolved_through_uic_code"] = False
    places["resolved_through_insee_code"] = False
    places["resolved_through_tvs_code"] = False
    places["resolved_through_iata_code"] = False
    places["resolved_through_gmap_api"] = False
    places["lon"] = None
    places["address"] = ""
    places["lat"] = None

    hardcode_resolver = HardcodesResolver()
    places = places.progress_apply(hardcode_resolver.resolve, axis=1)

    # Initiate geocoder
    GMAP_API_KEY = CONFIG["api_keys"]["gmap"]
    geocoding_api_resolver = GeocodingApiResolver(api_name="gmap", api_key=GMAP_API_KEY)

    # Apply the geocoder with delay using the rate limiter:
    places.loc[~places["resolved"], :] = places[~places["resolved"]].progress_apply(
        geocoding_api_resolver.resolve, axis=1
    )
    geocoding_api_resolver.save_cache()
    for c in places:
        if "resolved_through_" in c:
            print(c, places[c].sum())

    places["lat"] = places["lat"].astype(float)
    places["lon"] = places["lon"].astype(float)
    places_dict = places.set_index("place").to_dict()

    for place_index in [0, 1, 2]:
        trips["coords_place_%d_lat" % place_index] = trips[
            "trip_place_%d" % place_index
        ].apply(lambda x: places_dict["lat"][x])
        trips["coords_place_%d_lon" % place_index] = trips[
            "trip_place_%d" % place_index
        ].apply(lambda x: places_dict["lon"][x])
        trips["coords_place_%d" % place_index] = (
            trips["coords_place_%d_lon" % place_index].astype(str)
            + ";"
            + trips["coords_place_%d_lat" % place_index].astype(str)
        )
        trips["place_%d_count" % place_index] = trips[
            "trip_place_%d" % place_index
        ].apply(lambda x: places_dict["total"][x])

    trips["distance"] = trips.apply(compute_distance, axis=1)
    places.to_csv("./data/clean/places.csv", index=False)
    trips.to_csv("./data/clean/trips.csv", index=False)


/Users/gustaveronteix/opt/anaconda3/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


Prestation types: 
['T' 'TU' 'TCA' 'TC']
Unique places src:  1009
Unique places dst:  1001
Unique places stop:  1409
Unique places:  1668
Unique trips:  16574


NameError: name 'HardcodesResolver' is not defined

In [5]:
import numpy as np

data = dataset.load_data("./data/raw/sample-avion-train.csv")
places, trips = dataset.get_places_and_trips(data, ["A", 'AM', 'AU'])

Prestation types: 
['A' 'AU' 'AM']
Unique places src:  312
Unique places dst:  310
Unique places stop:  1
Unique places:  327
Unique trips:  1157


In [72]:
if __name__ == "__main__":
    data = dataset.load_data("./data/raw/sample-avion-train.csv")
    places, trips = dataset.get_places_and_trips(data, ["A", "AU", "AT"])

    # Splitting codes / name
    places[["code_1", "name", "temp", "code_2"]] = places["place"].str.extract(
        "([^-]*)\s-\s([^\[]*)\s?([\[](.*)[\]])?", expand=True
    )
    places.drop(columns=["temp"], inplace=True)

    places["resolved"] = False
    places["resolved_through_uic_code"] = False
    places["resolved_through_insee_code"] = False
    places["resolved_through_tvs_code"] = False
    places["resolved_through_iata_code"] = False
    places["resolved_through_gmap_api"] = False
    places["lon"] = None
    places["address"] = ""
    places["lat"] = None

    hardcode_resolver = HardcodesResolver()
    places = places.progress_apply(hardcode_resolver.resolve, axis=1)

Prestation types: 
['A' 'AU']
Unique places src:  288
Unique places dst:  285
Unique places stop:  1
Unique places:  300
Unique trips:  1036





100%|██████████| 300/300 [00:00<00:00, 2880.27it/s]


In [60]:
a = places.loc[0]
with open(("./data/prepared/iata_codes.pkl"), "rb") as file_id:
    iata_codes = dill.load(file_id)

In [65]:
'CDG' in iata_codes['lon']

True

In [70]:
a["code_1"][-3:] in iata_codes['lon']
a["lon"] = iata_codes["lon"][a["code_1"][-3:]]
a["lat"] = iata_codes["lat"][a["code_1"][-3:]]

In [ ]:
str(x["code_1"][-3:]) in self.iata_codes["lon"]:
                x["lon"] = self.iata_codes["lon"][x["code_1"]]
                x["lat"] = self.iata_codes["lat"][x["code_1"]]
                x["resolved"] = True

In [71]:
class HardcodesResolver:
    def __init__(self):
        self.insee_codes = self.load_codes("./data/prepared/insee_codes.pkl")
        self.tvs_codes = self.load_codes("./data/prepared/tvs_codes.pkl")
        self.uic_codes = self.load_codes("./data/prepared/uic_codes.pkl")
        self.iata_codes = self.load_codes("./data/prepared/iata_codes.pkl")

    def load_codes(self, path):
        with open(path, "rb") as file_id:
            return dill.load(file_id)

    def resolve(self, x):
        code_2_as_int = self.get_int(x["code_2"])
        
        try:
            if code_2_as_int and (code_2_as_int in self.uic_codes["lon"]):
                x["lon"] = self.uic_codes["lon"][code_2_as_int]
                x["lat"] = self.uic_codes["lat"][code_2_as_int]
                x["resolved"] = True
                x["resolved_through_uic_code"] = True
            elif x["code_1"] in self.insee_codes["lon"]:
                x["lon"] = self.insee_codes["lon"][x["code_1"]]
                x["lat"] = self.insee_codes["lat"][x["code_1"]]
                x["resolved"] = True
                x["resolved_through_insee_code"] = True
            elif str(x["code_1"][-3:]) in self.iata_codes["lon"]:                
                x["lon"] = self.iata_codes["lon"][x["code_1"][-3:]]
                x["lat"] = self.iata_codes["lat"][x["code_1"][-3:]]
                x["resolved"] = True
                x["resolved_through_iata_code"] = True
            elif x["place"] == '-75056': # Hard exception...
                x["name"] = 'Paris'
            # elif x['code_1'] in TVS_CODES['lon']:
            #     x['lon'] = TVS_CODES['lon'][x['code_1']]
            #     x['lat'] = TVS_CODES['lat'][x['code_1']]
            #     x['resolved'] = True
            #     x['resolved_through_tvs_code'] = True
            else:
                x["lon"] = None
                x["lat"] = None
                
        except:
            print(x["code_1"])
        return x

    def get_int(self, x):
        try:
            return int(x)
        except ValueError:
            return None